In [3]:
from gensim import corpora, models, similarities, downloader
import csv
import numpy as np

/Users/scavenger/opt/miniconda3/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [4]:
dataset = downloader.load('word2vec-google-news-300')
print(dataset)

In [5]:
# if len(dataset['king']) != 0:
#     print("hello")
row_pairs = []
lines = []
with open("synonyms.csv",'r') as test:
    csvreader = csv.reader(test)
    next(csvreader)
    #0: question, 1: answer, 2-5: choices
    for row in csvreader:
        lines.append(row)
        pairs = [] 
        for i in range(len(row)):
            if i < 2:
                continue
            pairs.append((row[0],row[i]))
        row_pairs.append(pairs)
with open('word2vec-google-news-300-details.csv', 'w', newline='') as new_file:
    csvwriter = csv.writer(new_file,delimiter=',')
    count = 0
    guess = False
    correct = True
    csvwriter.writerow(['Question','Answer','Guessed_Word','Result Label'])
    for pairs in row_pairs:
        cosine = []
        row = []
        for x,y in pairs:
            if x in dataset and y in dataset:
                cosine.append(dataset.similarity(x,y))
        row.append(lines[count][0])
        row.append(lines[count][1])
        if len(cosine) != 0:
            guessed_word = pairs[np.argmax(cosine)][1]
            row.append(guessed_word)
            if guessed_word == lines[count][1]:
                row.append('Correct')
            else:
                row.append('Wrong')
        else:
            row.append('Guess')
            row.append('Guess')
        csvwriter.writerow(row)
        count += 1

In [6]:
vocab_size = dataset.vectors.shape[0]
file = open("word2vec-google-news-300-details.csv", 'r')
read_data = file.read()
correct_count = read_data.lower().count("correct")
wrong_count = read_data.lower().count("wrong")
accuracy = (correct_count/(correct_count+wrong_count))

In [7]:
with open('analysis.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile,delimiter=',')
    csvwriter.writerow(['Model Name','Vocab Size','Correct Number','Wrong Number', 'Accuracy'])
    csvwriter.writerow(["word2vec-google-news-300-details.csv", vocab_size, correct_count, wrong_count, accuracy])